In [4]:
import polars as pl
import altair as alt

In [6]:
import warnings
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [8]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).filter(
    ~(pl.col("komora_komplet").str.contains("Senát") 
      | 
      pl.col('komora_komplet').str.contains('Společn') 
      | 
      pl.col('komora').str.contains('psse')
     )
).drop_nulls(
    subset=['datum']
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov")
).sort(
    by='datum'
).join(
    pl.scan_parquet(
        "data/projevy_jazyky.parquet",
    ),
    how="left",
    on="text"
).filter(
    pl.col("jazyk") != "sk"
).collect()

In [9]:
def hledej(retezec, doslova=False, frejm=df):
    return frejm.with_columns(
        pl.col('text').str.count_matches(retezec).alias('vyskytu')
    ).group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col('vyskytu').sum()
    ).join(
        frejm.group_by_dynamic(
            index_column='datum',every='1y'
        ).agg(
            pl.col('pocet_slov').sum()
        ), 
        on='datum', how='left'
    ).with_columns(
    (pl.col('vyskytu') / pl.col('pocet_slov')).alias('podil')
).with_columns(
        pl.lit(retezec).alias('co')
)

In [207]:
df.group_by("komora_komplet").len().sort(by="len",descending=True)

komora_komplet,len
str,u32
"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""",90906
"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""",83854
"""PČR, PS 2021-...""",82903
"""Parlament České republiky, Poslanecká sněmovna 1998 – 2002""",75999
"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""",70598
"""Parlament České republiky, Poslanecká sněmovna 2006 – 2010""",57117
"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""",46411
"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""",30803
"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""",29982


In [25]:
def graf(co):
    return alt.Chart(
    hledej(co).to_pandas(),
    width = 600
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

In [203]:
graf("Pra[hz][aeo]")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [29]:
graf("Brn[aouě]")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [31]:
graf("[Gg]ottwaldov[^\w]")

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_28436\4177337425.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf("[Gg]ottwaldov[^\w]")
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [33]:
graf('Zlín')

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [89]:
alt.Chart(
    hledej("Bečv").to_pandas(),
    width = 600
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [90]:
alt.Chart(
    hledej("Šlunkn").to_pandas(),
    width = 600
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [10]:
graf("[Bb]ratislav")

NameError: name 'graf' is not defined

In [37]:
graf("Ostrav")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [39]:
graf("Berlín")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [41]:
graf("Mosk[ev]")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [43]:
graf("Washingt")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [45]:
graf("Kábul")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [47]:
graf("Brusel")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [51]:
graf("Havl\w{1,5} Br")

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_28436\4008654840.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf("Havl\w{1,5} Br")
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [53]:
graf("Havířov")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [49]:
graf("Liber[ec]")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [55]:
graf("Víd(eň|n)")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [101]:
alt.Chart(
    hledej(r"[Ss]tředn\w{1,3}\sEvr").to_pandas(),
    width = 600
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [102]:
alt.Chart(
    hledej(r"[Vv]ýcho\w{1,3}\sEvr").to_pandas(),
    width = 600
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [103]:
alt.Chart(
    hledej(r"[Zz]ápa\w{1,3}\sEvr").to_pandas(),
    width = 600
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

## K počtu obyvatel

In [154]:
mesta_z_wikipedie = [
    ['Pra[hz]','1 397 880'],
    ['Brn[oaeuě]','402 739'],
    ['Ostrav','283 187'],
    ['Plz[en]','187 928'],
    ['Liber[ce]','108 090'],
    ['Olomouc','103 063'],
    ['Budějov','97 231'],
    ['Hrad\w{2,5}\sKrál','	94 311'],
    ['Pardubi','	92 319'],
    ['Ústí nad Lab','90 866'],
    ['(Zlín|Gottwaldov)','74 684'],
    ['Kladn[oaeuě]','69 664'],
    ['Havířo','68 674'],
    ['Opav','55 109'],
    ['Jihlav','54 624'],
    ['Frýd','	53 590'],
    ['Teplic','50 912'],
    ['Karvin','48 937'],
    ['Boleslav','47 346'],
    ['Chomutov','46 771'],
    ['Děčín','46 376'],
    ['Jablon','46 209'],
    ['Prostěj','43 408'],
    ['Přerov','40 906'],
    [r'Čes\w{1,5} Líp','36 815'],
    ['Třebíč','	34 530'],
    ['Znoj[em]','34 172'],
    ['Kolín','33 444'],
    ['Příbram','32 773'],
    ['Kroměříž','27 917'],
    ['Třin[ec]','33 852']
]
mesta = []

for m in mesta_z_wikipedie:
    mesta.append({'mesto':m[0],'obyvatel':int(m[1].replace(' ',''))})

<>:9: SyntaxWarning: invalid escape sequence '\w'
<>:9: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_28436\1595229417.py:9: SyntaxWarning: invalid escape sequence '\w'
  ['Hrad\w{2,5}\sKrál','	94 311'],


In [187]:
seznam_mest

['Pra[hz]',
 'Brn[oaeuě]',
 'Ostrav',
 'Plz[en]',
 'Liber[ce]',
 'Olomouc',
 'Budějov',
 'Hrad\\w{2,5}\\sKrál',
 'Pardubi',
 'Ústí nad Lab',
 '(Zlín|Gottwaldov)',
 'Kladn[oaeuě]',
 'Havířo',
 'Opav',
 'Jihlav',
 'Frýd',
 'Teplic',
 'Karvin',
 'Boleslav',
 'Chomutov',
 'Děčín',
 'Jablon',
 'Prostěj',
 'Přerov',
 'Čes\\w{1,5} Líp',
 'Třebíč',
 'Znoj[em]',
 'Kolín',
 'Příbram',
 'Kroměříž',
 'Třin[ec]']

In [144]:
soucasne_obdobi = df.filter(pl.col('obdobi') >= 1993)
prastare_obdobi = df.filter(pl.col('obdobi').is_between(1918,1940))
soc_obdobi = df.filter(pl.col('obdobi').is_between(1945,1989))

In [156]:
statistiky_mest = []

for m in mesta:
    statistiky_mest.append(
        {'mesto':m['mesto'],'zminek':len(soucasne_obdobi.filter(pl.col('text').str.contains(m['mesto']))),'obyvatel':m['obyvatel']}
    )

statistiky_mest = pl.DataFrame(statistiky_mest).with_columns(
    (pl.col('obyvatel') / pl.col('zminek')).alias('pomer')
).sort(by='pomer')

statistiky_mest

mesto,zminek,obyvatel,pomer
str,i64,i64,f64
"""(Zlín|Gottwaldov)""",1491,74684,50.089873
"""Olomouc""",1138,103063,90.565026
"""Pardubi""",859,92319,107.472643
"""Plz[en]""",1741,187928,107.942562
"""Opav""",449,55109,122.737194
"""Karvin""",388,48937,126.126289
"""Ostrav""",2219,283187,127.619198
"""Přerov""",320,40906,127.83125
"""Kroměříž""",215,27917,129.846512


In [148]:
statistiky_mest_kdysi = []

for m in mesta:
    statistiky_mest_kdysi.append(
        {'mesto':m['mesto'],'zminek':len(prastare_obdobi.filter(pl.col('text').str.contains(m['mesto']))),'obyvatel':m['obyvatel']}
    )

statistiky_mest_kdysi = pl.DataFrame(statistiky_mest_kdysi).with_columns(
    (pl.col('obyvatel') / pl.col('zminek')).alias('pomer')
).sort(by='pomer')

statistiky_mest_kdysi

mesto,zminek,obyvatel,pomer
str,i64,i64,f64
"""Jablon""",684,46209,67.557018
"""Opav""",732,55109,75.285519
"""Teplic""",518,50912,98.285714
"""Znoj[em]""",346,34172,98.763006
"""Liber[ce]""",784,108090,137.869898
"""Olomouc""",688,103063,149.800872
"""Děčín""",301,46376,154.07309
"""Kolín""",209,33444,160.019139
"""Brn[oaeuě]""",2409,402739,167.180988


In [149]:
statistiky_mest_soc = []

for m in mesta:
    statistiky_mest_soc.append(
        {'mesto':m['mesto'],'zminek':len(soc_obdobi.filter(pl.col('text').str.contains(m['mesto']))),'obyvatel':m['obyvatel']}
    )

statistiky_mest_soc = pl.DataFrame(statistiky_mest_soc).with_columns(
    (pl.col('obyvatel') / pl.col('zminek')).alias('pomer')
).sort(by='pomer')

statistiky_mest_soc

mesto,zminek,obyvatel,pomer
str,i64,i64,f64
"""Klad[en]""",418,69664,166.660287
"""(Zlín|Gottwaldov)""",416,74684,179.528846
"""Chomutov""",134,46771,349.037313
"""Třin[ec]""",92,33852,367.956522
"""Kroměříž""",66,27917,422.984848
"""Teplic""",113,50912,450.548673
"""Ostrav""",596,283187,475.145973
"""Kolín""",66,33444,506.727273
"""Přerov""",76,40906,538.236842


In [152]:
statistiky_mest_kdysi.join(
    statistiky_mest, how="left", on="mesto"
).with_columns(
    (pl.col("pomer") / pl.col("pomer_right")).alias("vyvoj")
).sort(
    by="vyvoj"
)

mesto,zminek,obyvatel,pomer,zminek_right,obyvatel_right,pomer_right,vyvoj
str,i64,i64,f64,i64,i64,f64,f64
"""Jablon""",684,46209,67.557018,116,46209,398.353448,0.169591
"""Teplic""",518,50912,98.285714,177,50912,287.638418,0.341699
"""Čes\w{1,5} Líp""",176,36815,209.176136,68,36815,541.397059,0.386364
"""Znoj[em]""",346,34172,98.763006,167,34172,204.622754,0.482659
"""Opav""",732,55109,75.285519,449,55109,122.737194,0.613388
"""Chomutov""",237,46771,197.345992,214,46771,218.556075,0.902954
"""Frýd""",282,53590,190.035461,261,53590,205.32567,0.925532
"""Děčín""",301,46376,154.07309,283,46376,163.872792,0.940199
"""Prostěj""",198,43408,219.232323,199,43408,218.130653,1.005051


In [106]:
statistiky_mest_soc.join(
    statistiky_mest, how="left", on="mesto"
).with_columns(
    (pl.col("pomer") / pl.col("pomer_right")).alias("vyvoj")
).sort(
    by="vyvoj"
)

mesto,zminek,obyvatel,pomer,zminek_right,obyvatel_right,pomer_right,vyvoj
str,i64,i64,f64,i64,i64,f64,f64
"""Jablon""",83,46209,556.73494,116,46209,398.353448,1.39759
"""Třin[ec]""",92,33852,367.956522,130,33852,260.4,1.413043
"""Teplic""",113,50912,450.548673,177,50912,287.638418,1.566372
"""Chomutov""",134,46771,349.037313,214,46771,218.556075,1.597015
"""Čes\w{1,5} Líp""",42,36815,876.547619,68,36815,541.397059,1.619048
"""Třebíč""",56,34530,616.607143,133,34530,259.62406,2.375
"""Ústí""",230,90866,395.069565,608,90866,149.450658,2.643478
"""Znoj[em]""",62,34172,551.16129,167,34172,204.622754,2.693548
"""Prostěj""",69,43408,629.101449,199,43408,218.130653,2.884058


In [108]:
statistiky_mest_soc.join(
    statistiky_mest_kdysi, how="left", on="mesto"
).with_columns(
    (pl.col("pomer") / pl.col("pomer_right")).alias("vyvoj")
).sort(
    by="vyvoj"
)

mesto,zminek,obyvatel,pomer,zminek_right,obyvatel_right,pomer_right,vyvoj
str,i64,i64,f64,i64,i64,f64,f64
"""Havířo""",41,68674,1674.97561,0,68674,inf,0.0
"""(Zlín|Gottwaldov)""",416,74684,179.528846,174,74684,429.218391,0.418269
"""Třin[ec]""",92,33852,367.956522,71,33852,476.788732,0.771739
"""Klad[en]""",418,69664,166.660287,409,69664,170.327628,0.978469
"""Třebíč""",56,34530,616.607143,98,34530,352.346939,1.75
"""Chomutov""",134,46771,349.037313,237,46771,197.345992,1.768657
"""Pardub""",121,92319,762.966942,218,92319,423.481651,1.801653
"""Karvin""",59,48937,829.440678,127,48937,385.330709,2.152542
"""Ostrav""",596,283187,475.145973,1319,283187,214.698256,2.213087


## Proměnlivost výskytu měst celkově

In [191]:
import re

def combine_regex_patterns(patterns):
    """Transform a list of regex patterns into a single OR pattern."""
    if not patterns:
        return ""
    return "(" + "|".join(patterns) + ")"

In [193]:
regex_mesta = combine_regex_patterns([x['mesto'] for x in mesta])

In [195]:
graf(regex_mesta)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

## Proměnlivost výskytu jednotlivých měst

In [140]:
statistiky_mest.with_columns(
    pl.col('pomer') / 32
).rename({"pomer":"po_1993"}).join(
    statistiky_mest_soc.with_columns(pl.col('pomer') / 45).select(pl.col(['pomer','mesto'])).rename({"pomer":"1945-1990"}),
    on="mesto",
    how="left"
).join(
    statistiky_mest_kdysi.with_columns(pl.col('pomer') / 20).select(pl.col(['pomer','mesto'])).rename({"pomer":"pred_1945"}),
    on="mesto",
    how="left"
).with_columns(
    pl.concat_list(["po_1993", "1945-1990", "pred_1945"]).list.var().alias("row_variance")
).sort(by="row_variance")

mesto,zminek,obyvatel,po_1993,1945-1990,pred_1945,row_variance
str,i64,i64,f64,f64,f64,f64
"""Chomutov""",214,46771,6.829877,7.756385,9.8673,2.423385
"""Ústí""",608,90866,4.670333,8.779324,7.559567,4.453214
"""Teplic""",177,50912,8.988701,10.012193,4.914286,7.272841
"""Hrad[ec]""",688,94311,4.283748,10.692857,9.783299,12.024838
"""Klad[en]""",1365,69664,1.594872,3.703562,8.516381,12.586183
"""Prostěj""",199,43408,6.816583,13.980032,10.961616,12.934524
"""Ostrav""",2219,283187,3.9881,10.558799,10.734913,14.787432
"""Znoj[em]""",167,34172,6.394461,12.248029,4.93815,14.969903
"""Brn[oaeuě]""",2845,402739,4.423759,12.676708,8.359049,17.039975


In [16]:
hledej('Pardub')

datum,vyskytu,pocet_slov,podil,co
date,u32,u32,f64,str
1918-01-01,1,172010,0.000006,"""Pardub"""
1919-01-01,12,1547459,0.000008,"""Pardub"""
1920-01-01,10,1557497,0.000006,"""Pardub"""
1921-01-01,24,1814119,0.000013,"""Pardub"""
1922-01-01,20,1739976,0.000011,"""Pardub"""
1923-01-01,9,1131296,0.000008,"""Pardub"""
1924-01-01,7,1386775,0.000005,"""Pardub"""
1925-01-01,7,989387,0.000007,"""Pardub"""
1926-01-01,26,1629693,0.000016,"""Pardub"""


In [22]:
alt.Chart(
    pl.concat(
        [
            hledej('Pardub'),
            hledej('Karvin'),
            hledej('Chomutov'),
            hledej('Teplic'),
            hledej(r"Brn[oaeuě]")
        ]
    ).to_pandas(),
    width = 600
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q'),
    alt.Row('co:N')
)

alt.Chart(...)

## Regiony

In [159]:
graf("(?i)morav")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [161]:
graf("(?i)slovensk")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [163]:
graf("Krkonoš")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [165]:
graf("Beskyd")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [167]:
graf("Šumav")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [169]:
graf("Podyjí")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [171]:
graf("Slováck")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [173]:
graf("Valaš")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [179]:
graf("Haná")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)